In [1]:
#Active state example NCAA hoops predictor script

In [2]:
import pandas as pd
import numpy as np

# import xgboost as xgb

# you can also use = glob.glob('../
#these files are from (https://www.kaggle.com/c/mens-march-mania-2022/data)

tcr_df = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/Basics/MNCAATourneyCompactResults.csv')
ts_df = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/Basics/MNCAATourneySeeds.csv')
rscr_df = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/Basics/MRegularSeasonCompactResults.csv')
ss_df = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/Basics/MSampleSubmissionStage1_2020.csv')
massey_df = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/PublicRankings/MMasseyOrdinals.csv')
tbs_df = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/TeamBoxScores/MNCAATourneyDetailedResults.csv')
rsbs_df = pd.read_csv('../google-cloud-ncaa-march-madness-2020-division-1-mens-tournament/TeamBoxScores/MRegularSeasonDetailedResults.csv')

# First we normalized the number of points scored in a game to account for games that go into overtime (otherwise, the points-per-game statistic is overestimated for the teams that play more overtimes). Then we averaged the points-per-game across every game in each season. We also calculated the number of games in each season and win percentage for the season.

In [3]:
def get_ppg(rscr_df):
    
    #Gametime normalization 
    rscr_df['GameDuration'] = rscr_df['NumOT']
    for i in range(len(rscr_df.NumOT.value_counts())):
        rscr_df.loc[rscr_df['NumOT'] == rscr_df.NumOT.value_counts().index[i], 'GameDuration'] = 40 + i*5
    rscr_df['Wppg'] = (rscr_df['WScore'] / rscr_df['GameDuration']) * 40
    rscr_df['Lppg'] = (rscr_df['LScore'] / rscr_df['GameDuration']) * 40
    
    #Drop unnecesary columns 
    rscr_df = rscr_df.drop(['DayNum', 'WLoc', 'NumOT', 'WScore','LScore','GameDuration'], axis = 1)
    
    #Calculate average points per game and number of games for winning and losing team
    Wppg_df = rscr_df[["Season","WTeamID",'Wppg']].groupby(["Season","WTeamID"]).agg(['mean', 'count'])
    Lppg_df = rscr_df[["Season","LTeamID",'Lppg']].groupby(["Season","LTeamID"]).agg(['mean', 'count'])
    Wppg_df.columns = Wppg_df.columns.droplevel(0)
    Wppg_df = Wppg_df.reset_index().rename(columns = {'mean':'Wppg', 'count':'WGames'})
    Lppg_df.columns = Lppg_df.columns.droplevel(0)
    Lppg_df = Lppg_df.reset_index().rename(columns = {'mean':'Lppg', 'count':'LGames'})
    
    #Merge PPG for wins and losses and fill NaNs with 0s for undefeated and no-win teams
    ppg_df = pd.merge(Wppg_df, Lppg_df, left_on=['Season', 'WTeamID'], right_on=['Season', 'LTeamID'], how='outer')
    ppg_df = ppg_df.fillna({'WTeamID': ppg_df.LTeamID, 'Wppg': 0, 'WGames': 0, 'LTeamID': ppg_df.WTeamID , 'Lppg': 0, 'LGames': 0})
    
    #Calculate PPG and win percentage for season 
    ppg_df['PPG'] = (ppg_df['Wppg']*ppg_df['WGames'] + ppg_df['Lppg']*ppg_df['LGames'])/(ppg_df['WGames'] + ppg_df['LGames'])
    ppg_df['WPerc'] = (ppg_df['WGames'])/(ppg_df['WGames'] + ppg_df['LGames'])
    ppg_df['TeamID'] = ppg_df['WTeamID'].astype('int32')
    ppg_df['GamesPlayed'] = (ppg_df['WGames'] + ppg_df['LGames'])
    ppg_df = ppg_df.drop(['WTeamID', 'Wppg' ,'WGames','LTeamID', 'Lppg','LGames'],axis = 1)
    
    return ppg_df 

In [4]:
rscr_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0
...,...,...,...,...,...,...,...,...
161547,2019,132,1153,69,1222,57,N,0
161548,2019,132,1209,73,1426,64,N,0
161549,2019,132,1277,65,1276,60,N,0
161550,2019,132,1387,55,1382,53,N,0


# In the function below, I created an offensive metric based on points made, assists, and offensive rebounds, as well as a defensive metric based on opposing team points missed, steals, defensive rebounds, turnovers forced, blocks, and personal fouls. I averaged the metrics across the entire regular season, and also the last 30 days of the season.

In [5]:
def get_efficiency(rsbs_df):
    #Gametime normalization 
    rsbs_df['GameDuration'] = rsbs_df['NumOT']
    for i in range(len(rsbs_df.NumOT.value_counts())):
        rsbs_df.loc[rsbs_df['NumOT'] == rsbs_df.NumOT.value_counts().index[i], 'GameDuration'] = 40 + i*5

    rsbs_df['WFGM'] = (rsbs_df['WFGM'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WFGA'] = (rsbs_df['WFGA'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WFGM3'] = (rsbs_df['WFGM3'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WFGA3'] = (rsbs_df['WFGA3'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WFTM'] = (rsbs_df['WFTM'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WFTA'] = (rsbs_df['WFTA'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WOR'] = (rsbs_df['WOR'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WDR'] = (rsbs_df['WDR'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WAst'] = (rsbs_df['WAst'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WTO'] = (rsbs_df['WTO'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WStl'] = (rsbs_df['WStl'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WBlk'] = (rsbs_df['WBlk'] / rsbs_df['GameDuration']) * 40
    rsbs_df['WPF'] = (rsbs_df['WPF'] / rsbs_df['GameDuration']) * 40

    rsbs_df['LFGM'] = (rsbs_df['LFGM'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LFGA'] = (rsbs_df['LFGA'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LFGM3'] = (rsbs_df['LFGM3'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LFGA3'] = (rsbs_df['LFGA3'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LFTM'] = (rsbs_df['LFTM'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LFTA'] = (rsbs_df['LFTA'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LOR'] = (rsbs_df['LOR'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LDR'] = (rsbs_df['LDR'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LAst'] = (rsbs_df['LAst'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LTO'] = (rsbs_df['LTO'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LStl'] = (rsbs_df['LStl'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LBlk'] = (rsbs_df['LBlk'] / rsbs_df['GameDuration']) * 40
    rsbs_df['LPF'] = (rsbs_df['LPF'] / rsbs_df['GameDuration']) * 40
    
    #Calculate total points made and missed per game 
    rsbs_df['wPointsMade'] = (2* (rsbs_df['WFGM'] - rsbs_df['WFGM3'])) + 3*rsbs_df['WFGM3'] + rsbs_df['WFTM']
    rsbs_df['wPointsMissed'] = (2* (rsbs_df['WFGA'] - rsbs_df['WFGA3'])) + 3*rsbs_df['WFGA3'] + rsbs_df['WFTA'] - rsbs_df['wPointsMade']
    rsbs_df['lPointsMade'] = (2* (rsbs_df['LFGM'] - rsbs_df['LFGM3'])) + 3*rsbs_df['LFGM3'] + rsbs_df['LFTM']
    rsbs_df['lPointsMissed'] = (2* (rsbs_df['LFGA'] - rsbs_df['LFGA3'])) + 3*rsbs_df['LFGA3'] + rsbs_df['LFTA'] - rsbs_df['lPointsMade']
    
    #Calculate offensive and defensive efficiency metrics for winning and losing team 
    rsbs_df['WoEFF'] = rsbs_df['wPointsMade'] +  rsbs_df['WAst'] + rsbs_df['WOR']
    rsbs_df['WdEFF'] = rsbs_df['lPointsMissed'] + rsbs_df['WStl'] + rsbs_df['WBlk'] + rsbs_df['WDR'] + rsbs_df['LTO'] - rsbs_df['WPF']
    rsbs_df['LoEFF'] = rsbs_df['lPointsMade'] +  rsbs_df['LAst'] + rsbs_df['LOR']
    rsbs_df['LdEFF'] = rsbs_df['wPointsMissed'] + rsbs_df['LStl'] + rsbs_df['LBlk'] + rsbs_df['LDR'] + rsbs_df['WTO'] - rsbs_df['LPF']
    
    #Extract relevant columns and rename 
    Weff_df = rsbs_df[['Season', 'DayNum', 'WTeamID','WoEFF', 'WdEFF']]
    Weff_df = Weff_df.rename(columns = {'WTeamID': 'TeamID','WoEFF': 'oEFF', 'WdEFF': 'dEFF'})
    Leff_df = rsbs_df[['Season', 'DayNum', 'LTeamID','LoEFF', 'LdEFF']]
    Leff_df = Leff_df.rename(columns = {'LTeamID': 'TeamID','LoEFF': 'oEFF', 'LdEFF': 'dEFF'})
    eff_df = pd.concat([Weff_df, Leff_df]) 
    
    #Take seasonal and 30 day averages of the efficiency metrics
    # the 30-day efficiency metrics more accurately capture how ‘hot’ a team is, going into the tournament.
    
    effseas_df = eff_df.groupby(by = ['Season', 'TeamID']).agg('mean').reset_index().drop(['DayNum'], axis = 1)
    eff30day_df = eff_df[eff_df.DayNum >= 100].groupby(by = ['Season', 'TeamID']).agg('mean').reset_index().drop(['DayNum'], axis = 1)
    eff30day_df = eff30day_df.rename(columns = {'oEFF': 'oEFF_30day', 'dEFF': 'dEFF_30day' })
    eff_df = pd.merge(effseas_df,eff30day_df, how = 'outer', on = ['Season', 'TeamID']) 
    return eff_df 

In [6]:
rsbs_df

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,WFGM,WFGA,...,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1104,68,1328,62,N,0,27,58,...,10,16,22,10,22,8,18,9,2,20
1,2003,10,1272,70,1393,63,N,0,26,62,...,24,9,20,20,25,7,12,8,6,16
2,2003,11,1266,73,1437,61,N,0,24,58,...,26,14,23,31,22,9,12,2,5,23
3,2003,11,1296,56,1457,50,N,0,18,38,...,22,8,15,17,20,9,19,4,3,23
4,2003,11,1400,77,1208,71,N,0,30,61,...,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87499,2019,132,1153,69,1222,57,N,0,22,50,...,33,11,18,17,16,8,7,2,4,19
87500,2019,132,1209,73,1426,64,N,0,20,50,...,33,11,17,13,28,12,14,5,2,24
87501,2019,132,1277,65,1276,60,N,0,22,55,...,25,10,12,3,26,17,6,5,5,11
87502,2019,132,1387,55,1382,53,N,0,22,59,...,19,8,10,13,30,9,11,2,7,16


# Because we’re interested in the ranking right before the tournament would have begun, we only extracted rankings with the RankingDayNum column equal to 133 (corresponding to the last day of the regular season)

In [7]:
def get_rank(massey_df):
    rankings_df = massey_df[massey_df['RankingDayNum'] == 133]
    rankings_df = rankings_df.reset_index().drop(['index','RankingDayNum'], axis = 1)
    median_df = rankings_df.groupby(by = ['Season','TeamID'])[['OrdinalRank']].median().reset_index()
    median_df = median_df.rename(columns = {'OrdinalRank':'MedianRank'})
    mean_df = rankings_df.groupby(by = ['Season','TeamID'])[['OrdinalRank']].mean().reset_index()
    mean_df = mean_df.rename(columns = {'OrdinalRank':'MeanRank'})
    massey_df = rankings_df[rankings_df['SystemName'] == 'MAS']
    massey_df = massey_df.reset_index().drop(['index','SystemName'], axis = 1)
    massey_df = massey_df.rename(columns = {'OrdinalRank':'MasseyRank'})
    rankings_df = pd.merge(median_df, massey_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')
    rankings_df = pd.merge(rankings_df, mean_df, left_on=['Season', 'TeamID'], right_on=['Season', 'TeamID'], how='left')
    return rankings_df 

# Before doing that, we should construct a dataframe with the correct submission structure to merge our features onto. Thankfully, Kaggle provides a sample submission file that we can use as a template. The first few rows of the sample look like the following:


Where each row represents a matchup in the tournament for each season. The ID consists of the season, the lower Team ID, and the higher Team ID. The Pred column corresponds to the predicted probability of the lower Team ID beating the higher Team ID. Within the context of our model, we used the features we engineered to predict this column. Then, we compared the prediction to the actual result.

In [8]:
def prep_submission(ss_df,tcr_df):
    features_df = ss_df.copy()
    
    #extract season and team IDs
    features_df['Season'] = features_df['ID'].map(lambda x: int(x[:4]))
    features_df['WTeamID'] = features_df['ID'].map(lambda x: int(x[5:9]))
    features_df['LTeamID'] = features_df['ID'].map(lambda x: int(x[10:14]))
    features_df = features_df.rename(columns= {'WTeamID':'TeamID_1','LTeamID':'TeamID_2'})
    
    #Extract seasons starting from 2015 and create Result column corresponding to winner and loser
    Wtcr_df = tcr_df[tcr_df.Season >= 2015].loc[:,['Season', 'WTeamID', 'LTeamID']].reset_index().drop('index', axis = 1)
    Wtcr_df = Wtcr_df.rename(columns = {'WTeamID': 'TeamID_1', 'LTeamID': 'TeamID_2'})
    Wtcr_df['Result'] = 1
    Ltcr_df = tcr_df[tcr_df.Season >= 2015].loc[:,['Season', 'WTeamID', 'LTeamID']].reset_index().drop('index', axis = 1)
    Ltcr_df = Ltcr_df.rename(columns = {'WTeamID': 'TeamID_2', 'LTeamID': 'TeamID_1'})
    Ltcr_df['Result'] = 0
    tcr_df = pd.concat([Wtcr_df,Ltcr_df], sort=False).reset_index().drop('index', axis = 1)
    tcr_df['ID'] = tcr_df['Season'].apply(str) + '_' + tcr_df['TeamID_1'].apply(str) + '_' + tcr_df['TeamID_2'].apply(str)
    tcr_df = tcr_df.drop(['Season', 'TeamID_1', 'TeamID_2'],axis = 1)
    
    #merge results onto sample submission
    features_df = pd.merge(features_df,tcr_df, how = 'left', on = 'ID')
    features_df = features_df.drop(['Pred'],axis = 1)
    return features_df 

# Besides generating a sample submission file, the above function also creates our test set, which corresponds to all possible tournament matchups for each of the past season’s tournaments. The output looks like this:

# We can create our training set (regular season games) in much the same way:

In [9]:
def prep_regseason(rscr_df):
    
    #Extract seasons starting from 2015 and create Result column corresponding to winner and loser
    
    Wrscr_df = rscr_df[rscr_df.Season >= 2015].loc[:,['Season', 'WTeamID', 'LTeamID']].reset_index().drop('index', axis = 1)
    Wrscr_df = Wrscr_df.rename(columns = {'WTeamID': 'TeamID_1', 'LTeamID': 'TeamID_2'})
    Wrscr_df['Result'] = 1
    Lrscr_df = rscr_df[rscr_df.Season >= 2015].loc[:,['Season', 'WTeamID', 'LTeamID']].reset_index().drop('index', axis = 1)
    Lrscr_df = Lrscr_df.rename(columns = {'WTeamID': 'TeamID_2', 'LTeamID': 'TeamID_1'})
    Lrscr_df['Result'] = 0
    rscr_df = pd.concat([Wrscr_df,Lrscr_df], sort=False).reset_index().drop('index', axis = 1)
    rscr_df['ID'] = rscr_df['Season'].apply(str) + '_' + rscr_df['TeamID_1'].apply(str) + '_' + rscr_df['TeamID_2'].apply(str)
    return rscr_df 

# Next we got on with merging the engineered features onto both the test and training sets. The function works equally well for both the training and test sets, with the only difference being the first argument of the function:

For the test set we used the output of the prep_submission() function.
For the training set we used the output of the prep_regseason() function.

In [10]:
def merge_features(features_df, ppg_df,eff_df,rankings_df):
    
    Wppg_df = ppg_df.rename(columns = {'PPG':'PPG_1','WPerc':'WPerc_1'})
    Lppg_df = ppg_df.rename(columns = {'PPG':'PPG_2','WPerc':'WPerc_2'})
    Wfeatures_df = pd.merge(features_df, Wppg_df,left_on=['Season', 'TeamID_1'], right_on=['Season', 'TeamID'], how='left')
    Wfeatures_df = Wfeatures_df.drop(['TeamID', 'GamesPlayed'], axis = 1)
    features_df = pd.merge(Wfeatures_df, Lppg_df,left_on=['Season', 'TeamID_2'], right_on=['Season', 'TeamID'], how='left')
    features_df = features_df.drop(['TeamID', 'GamesPlayed'], axis = 1)

    Weff_df = eff_df.rename(columns = {'oEFF': 'oEFF_1', 'dEFF': 'dEFF_1','oEFF_30day': 'oEFF_30day_1', 'dEFF_30day': 'dEFF_30day_1'})
    Leff_df = eff_df.rename(columns = {'oEFF': 'oEFF_2', 'dEFF': 'dEFF_2','oEFF_30day': 'oEFF_30day_2', 'dEFF_30day': 'dEFF_30day_2'})
    features_df = pd.merge(features_df, Weff_df, how = 'left', left_on = ['Season', 'TeamID_1'], right_on = ['Season', 'TeamID'])
    features_df = features_df.drop(['TeamID'], axis = 1)
    features_df = pd.merge(features_df, Leff_df, how = 'left', left_on = ['Season', 'TeamID_2'], right_on = ['Season', 'TeamID'])
    features_df = features_df.drop(['TeamID'], axis = 1)
    features_df['PPG_diff'] = features_df['PPG_1'] - features_df['PPG_2']
    features_df['WPerc_diff'] = features_df['WPerc_1'] - features_df['WPerc_2']
    features_df['oEFF_diff'] = features_df['oEFF_1'] - features_df['oEFF_2']
    features_df['dEFF_diff'] = features_df['dEFF_1'] - features_df['dEFF_2']
    features_df['oEFF_30day_diff'] = features_df['oEFF_30day_1'] - features_df['oEFF_30day_2']
    features_df['dEFF_30day_diff'] = features_df['dEFF_30day_1'] - features_df['dEFF_30day_2']
    features_df = features_df.drop(['PPG_1', 'WPerc_1', 'PPG_2', 'WPerc_2', 'oEFF_1', 'dEFF_1', 'oEFF_30day_1', 
                                    'dEFF_30day_1', 'oEFF_2', 'dEFF_2', 'oEFF_30day_2', 'dEFF_30day_2'], axis = 1)
    
    Wrankings_df = rankings_df.rename(columns = {'MedianRank': 'MedianRank_1', 'MasseyRank': 'MasseyRank_1', 'MeanRank' : 'MeanRank_1'})
    Lrankings_df = rankings_df.rename(columns = {'MedianRank': 'MedianRank_2', 'MasseyRank': 'MasseyRank_2', 'MeanRank' : 'MeanRank_2'})
    features_df = pd.merge(features_df,Wrankings_df, how = 'left', left_on = ['Season', 'TeamID_1'], right_on = ['Season', 'TeamID'])
    features_df = features_df.drop(['TeamID'],axis = 1)
    
    features_df = pd.merge(features_df,Lrankings_df, how = 'left', left_on = ['Season', 'TeamID_2'], right_on = ['Season', 'TeamID'])
    
    features_df = features_df.drop(['TeamID'],axis = 1)
    features_df['MedianRank_diff'] = features_df['MedianRank_1'] - features_df['MedianRank_2']
    features_df['MasseyRank_diff'] = features_df['MasseyRank_1'] - features_df['MasseyRank_2']
    features_df['MeanRank_diff'] = features_df['MeanRank_1'] - features_df['MeanRank_2']
    
    features_df = features_df.drop(['MedianRank_1', 'MasseyRank_1', 'MeanRank_1',
                                    'MedianRank_2', 'MasseyRank_2', 'MeanRank_2'], axis = 1)
    
    return features_df 

# The first part of the function merged the points-per-game dataframe, the efficiency metrics dataframe, and the rankings dataframe onto the respective test or training dataframe. This implies two sets of metrics in each row: one for Team 1 and another for Team 2. To reduce the number of features, but still maintain the same amount of information, we were able to take the difference between each metric for each team. So, for example, the column PPG_diff contains the difference in PPG between Team 1 and Team 2. This is done for each feature.

# Thus far, we have imported the data provided by the competition, and created functions for engineered features using this data. Now, we can move on to training the model.

# Training & Testing The Model: 2015 To 2019 Regular Season
After writing all of the data manipulation functions, we can run the imported data through each one:

In [11]:
## prepare tournament submission df for 2015-2019, i.e. test set. 
## Note: not all matchups have Results, as we predict all possible matchups (11,390) for the 5 seasons when only 335 games actually occur
features_df = prep_submission(ss_df,tcr_df)
ppg_df = get_ppg(rscr_df)
eff_df = get_efficiency(rsbs_df)
rankings_df = get_rank(massey_df)
tournament_df = merge_features(features_df, ppg_df,eff_df,rankings_df)

## prepare regular season df for 2015-2019, i.e. training set
rscr_df = prep_regseason(rscr_df)
regseason_df = merge_features(rscr_df, ppg_df,eff_df,rankings_df) 

# Tournament_df contains our test data and regseason_df contains our training data.

Next, I split each set into each season. This step is more a matter of preference and is not absolutely necessary. I prefer to have different model parameters for each season than to group them all together. This gives insight on how the evaluation metric can change from year to year,

In [12]:
regseason_df_2015 = regseason_df[regseason_df.Season == 2015]
regseason_df_2016 = regseason_df[regseason_df.Season == 2016]
regseason_df_2017 = regseason_df[regseason_df.Season == 2017]
regseason_df_2018 = regseason_df[regseason_df.Season == 2018]
regseason_df_2019 = regseason_df[regseason_df.Season == 2019] 

In [13]:
## drop matchups that don’t occur, as we cannot evaluate them
tournament_df = tournament_df.dropna()

tournament_df_2015 = tournament_df[tournament_df.Season == 2015]
tournament_df_2016 = tournament_df[tournament_df.Season == 2016]
tournament_df_2017 = tournament_df[tournament_df.Season == 2017]
tournament_df_2018 = tournament_df[tournament_df.Season == 2018]
tournament_df_2019 = tournament_df[tournament_df.Season == 2019] 

# Now we split the datasets into a test and training set. I only included the 2015 season below, but the code is identical for the other seasons (just swap the year in the dataframe).

In [14]:
X_train = regseason_df_2015[['PPG_diff', 'WPerc_diff', 'oEFF_diff', 'dEFF_diff', 'oEFF_30day_diff',
                       'dEFF_30day_diff', 'MedianRank_diff', 'MasseyRank_diff', 'MeanRank_diff']]
y_train = regseason_df_2015[['Result']] 
X_test = tournament_df_2015[['PPG_diff', 'WPerc_diff', 'oEFF_diff', 'dEFF_diff', 'oEFF_30day_diff',
                        'dEFF_30day_diff', 'MedianRank_diff', 'MasseyRank_diff', 'MeanRank_diff']]
y_test = tournament_df_2015[['Result']]  

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss

clf = LogisticRegression(penalty='l1', dual=False, tol=0.001, C=1.0, fit_intercept=True, 
                   intercept_scaling=1, class_weight='balanced', random_state=None, 
                   solver='liblinear', max_iter=1000, multi_class='ovr', verbose=0)
#train and predict
clf.fit(X_train, np.ravel(y_train.values))
y_pred = clf.predict_proba(X_test)
y_pred = y_pred[:,1]

#log loss scoring 
log_loss(y_test, y_pred, eps=1e-15, normalize=True, sample_weight=None, labels=None) 

0.5716522307172726

In [16]:
#import xgboost as xgb

#dtest = xgb.DMatrix(X_test, y_test, feature_names=X_test.columns)
#dtrain = xgb.DMatrix(X_train, y_train,feature_names=X_train.columns)

#param = {'verbosity':1, 
#         'objective':'binary:logistic',
#         'booster':'gblinear',
#         'eval_metric' :'logloss',
#         'learning_rate': 0.05}

#evallist = [(dtrain, 'train')] 

# Feel free to adjust the learning rate or any of the other parameters to see how they affect the final log loss value.

In [17]:
# num_round = 30
# bst = xgb.train(param, dtrain, num_round, evallist) 